<a href="https://colab.research.google.com/github/Debayan2004/TechnoHacksOfficial/blob/main/AnalyseWeatherData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d selfishgene/historical-hourly-weather-data

Dataset URL: https://www.kaggle.com/datasets/selfishgene/historical-hourly-weather-data
License(s): DbCL-1.0
 75% 9.00M/12.0M [00:00<00:00, 17.9MB/s]
100% 12.0M/12.0M [00:00<00:00, 24.5MB/s]


In [2]:
import zipfile
import os

# Define the path to the zip file and the directory to extract to
zip_file_path = '/content/historical-hourly-weather-data.zip'
extract_dir = '/content/'

# Create the extract directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Extracted all files to {extract_dir}')


Extracted all files to /content/


In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [44]:
# Load datasets
city_attributes = pd.read_csv('/content/city_attributes.csv')
humidity = pd.read_csv('/content/humidity.csv')
pressure = pd.read_csv('/content/pressure.csv')
temperature = pd.read_csv('/content/temperature.csv')
weather_description = pd.read_csv('/content/weather_description.csv')
wind_direction = pd.read_csv('/content/wind_direction.csv')
wind_speed = pd.read_csv('/content/wind_speed.csv')

In [25]:
# List of cities
cities = list(humidity.columns[1:])

In [26]:
# Initialize all_data DataFrame
all_data = pd.DataFrame()

In [27]:
# Iterate over each city and concatenate data
for city in cities:
    data = pd.DataFrame()
    data['datetime'] = humidity['datetime']
    data['humidity'] = humidity[city]
    data['temperature'] = temperature[city]
    data['wind_speed'] = wind_speed[city]
    data['city'] = city

In [29]:
# Handling missing values
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)

# Convert datetime to datetime type
data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime', inplace=True)

# Append to the all_data dataframe
all_data = pd.concat([all_data, data])

<ipython-input-29-30490f51a62e>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
<ipython-input-29-30490f51a62e>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [30]:
# Reset index
all_data.reset_index(inplace=True)

# Use a subset of the data (e.g., 10% of the data)
subset_fraction = 0.1
all_data = all_data.sample(frac=subset_fraction, random_state=42)

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
# Store city column separately to include it later
city_col = all_data['city'].map({city: i for i, city in enumerate(cities)}).values.reshape(-1, 1)
# Fit and transform the scaler without the city column
scaled_data = scaler.fit_transform(all_data.drop(columns=['datetime', 'city']))

# Concatenate the city column back after scaling
scaled_data = np.hstack([scaled_data, city_col])

In [31]:
city_attributes.describe()

,Latitude,Longitude
count,36.000000,36.000000
mean,37.066743,-73.544668
std,5.815514,51.612349
min,25.774269,-123.119339
25%,32.766126,-105.401312
50%,36.170429,-86.471241
75%,40.998211,-74.874332
max,49.249660,35.216331


In [32]:
humidity.describe()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,43427.000000,44804.000000,44311.000000,44964.000000,45101.000000,44909.000000,44411.000000,43945.000000,44543.000000,43445.000000,...,44629.000000,43629.000000,43557.000000,44804.000000,44394.000000,44140.000000,44283.000000,44435.000000,44436.000000,44347.000000
mean,81.895480,74.697616,76.875042,77.159038,62.773841,67.784809,31.937831,37.484424,45.186157,53.022557,...,68.017769,66.642417,71.861538,77.375301,70.604857,66.861509,53.155184,79.800383,78.606760,68.732293
std,14.522221,19.042656,17.396016,18.147464,21.818042,19.419307,20.041855,21.662728,23.336546,23.905392,...,18.790524,19.874727,16.825027,18.750190,21.321606,16.464177,27.305008,23.051692,23.682244,19.273881
min,12.000000,10.000000,6.000000,13.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,10.000000,10.000000,7.000000,11.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
25%,73.000000,63.000000,68.000000,66.000000,48.000000,56.000000,16.000000,21.000000,26.000000,33.000000,...,54.000000,51.000000,60.000000,65.000000,54.000000,58.000000,31.000000,63.000000,61.000000,56.000000
50%,86.000000,80.000000,81.000000,81.000000,66.000000,71.000000,27.000000,32.000000,42.000000,52.000000,...,68.000000,68.000000,74.000000,81.000000,77.000000,69.000000,48.000000,89.000000,87.000000,70.000000
75%,93.000000,90.000000,89.000000,93.000000,81.000000,82.000000,43.000000,50.000000,63.000000,73.000000,...,84.000000,83.000000,86.000000,93.000000,88.000000,78.000000,75.000000,100.000000,100.000000,83.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [33]:
pressure.describe()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,41019.000000,45249.000000,44438.000000,45240.000000,45001.000000,45078.000000,45165.000000,44659.000000,44797.000000,44710.000000,...,45249.000000,44212.000000,44259.000000,45246.000000,44447.000000,44323.000000,44461.000000,44455.000000,44458.000000,44454.000000
mean,1018.130915,1019.268382,1018.843872,1021.001216,1015.912735,1017.524491,1013.659648,1013.297342,1012.292185,1011.976359,...,1020.250193,1017.018977,1016.424953,1019.861314,991.172565,1013.815468,1002.039248,1017.550804,1015.901997,1011.034980
std,15.792425,9.943136,7.333477,9.737187,11.289701,8.554904,18.893592,11.988782,39.049838,41.343487,...,9.507596,10.125738,9.534390,9.548220,14.165100,6.476167,22.610353,7.500017,10.238495,12.951002
min,807.000000,968.000000,985.000000,980.000000,886.000000,964.000000,901.000000,884.000000,810.000000,800.000000,...,824.000000,959.000000,822.000000,978.000000,957.000000,929.000000,936.000000,999.000000,980.000000,981.000000
25%,1012.000000,1014.000000,1014.000000,1015.000000,1013.000000,1014.000000,1010.000000,1009.000000,1015.000000,1014.000000,...,1014.000000,1012.000000,1011.000000,1014.000000,985.000000,1010.000000,1002.000000,1012.000000,1011.000000,1002.000000
50%,1017.000000,1019.000000,1018.000000,1020.000000,1016.000000,1017.000000,1015.000000,1014.000000,1021.000000,1020.000000,...,1019.000000,1017.000000,1016.000000,1019.000000,988.000000,1013.000000,1010.000000,1017.000000,1017.000000,1012.000000
75%,1022.000000,1025.000000,1023.000000,1027.000000,1021.000000,1022.000000,1022.000000,1020.000000,1025.000000,1025.000000,...,1026.000000,1023.000000,1022.000000,1026.000000,994.000000,1018.000000,1016.000000,1024.000000,1023.000000,1020.000000
max,1100.000000,1056.000000,1044.000000,1058.000000,1044.000000,1043.000000,1047.000000,1076.000000,1054.000000,1089.000000,...,1059.000000,1052.000000,1054.000000,1054.000000,1099.000000,1039.000000,1040.000000,1042.000000,1042.000000,1042.000000


In [34]:
temperature.describe()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,44458.000000,45252.000000,44460.000000,45250.000000,45250.000000,45252.000000,45252.000000,45250.000000,45252.000000,45252.000000,...,45250.000000,44460.000000,45250.000000,45250.000000,44455.000000,44460.000000,44461.000000,44455.000000,44456.000000,44460.000000
mean,283.862654,284.992929,288.155821,284.409626,290.846116,290.215044,292.424887,295.493358,285.617856,282.839407,...,285.374168,285.400406,280.343010,283.779823,291.521986,294.512307,296.497276,295.266398,294.094803,293.184253
std,6.640131,7.452438,5.332862,6.547986,6.460823,5.889992,10.829522,9.916743,9.853484,10.673464,...,10.242377,10.220932,11.953626,9.802499,7.821815,6.676412,8.852984,6.324566,6.304118,7.093583
min,245.150000,262.370000,272.300000,263.780000,266.503667,265.783333,260.561333,266.059000,255.042333,247.727667,...,250.390000,250.774000,243.300000,249.540000,272.179000,271.049000,271.150000,271.150000,268.682000,272.974000
25%,279.160000,279.850000,284.670000,279.830000,286.380000,286.254750,283.920000,287.680000,277.970000,275.060000,...,277.350636,277.370000,271.971750,276.090000,285.366623,289.450000,289.734000,290.467333,289.881833,287.524279
50%,283.450000,284.320000,287.610000,283.940000,290.530000,290.118750,292.027486,295.586667,286.120000,282.970667,...,285.927583,285.870000,281.109000,284.133250,290.932667,294.900000,296.150000,294.820000,294.150000,292.996000
75%,288.600785,289.451750,291.015167,288.530000,295.080000,294.107542,300.835000,303.050000,292.835643,290.680000,...,293.796000,293.760000,290.369583,291.620000,297.270000,299.800000,303.150000,299.660000,298.930000,299.150000
max,307.000000,312.520000,313.620000,307.300000,315.470000,313.360000,318.640000,321.220000,312.710000,313.750000,...,308.000000,310.240000,307.880000,308.000000,314.820000,320.930000,320.150000,320.930000,313.150000,317.040000


In [35]:
weather_description.describe()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,45253,44460,45252,44460,45252,45252,45252,45252,45252,45252,...,45252,44460,45252,45252,44460,44460,44461,44460,44460,44460
unique,45253,37,24,28,29,25,22,23,26,37,...,30,36,37,27,8,26,20,20,20,23
top,2017-11-30 00:00:00,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,...,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear
freq,1,12805,11725,12654,12801,26136,14829,35090,30303,18344,...,14919,11601,8151,17444,29442,21739,37320,25587,25516,26934


In [36]:
wind_speed.describe()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,44458.000000,45252.000000,44459.000000,45252.000000,45252.000000,45252.000000,45238.000000,45251.000000,45249.000000,45251.000000,...,45251.000000,44460.000000,45252.000000,45252.000000,44460.000000,44459.000000,44461.000000,44458.000000,44459.000000,44459.000000
mean,2.432746,2.050738,2.786702,2.118116,1.219548,1.751458,2.463482,1.909063,2.763685,2.438996,...,2.643212,3.210954,3.831897,3.380889,1.989339,2.722306,3.452576,3.299406,3.024292,1.880047
std,2.004636,1.840815,2.284011,1.629988,1.434552,1.489450,2.318692,1.623629,2.300933,2.158956,...,1.836347,2.189716,2.483430,2.277266,1.462556,1.923380,2.323312,2.177502,2.124482,1.712640
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000
50%,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,...,2.000000,3.000000,3.000000,3.000000,1.000000,2.000000,3.000000,3.000000,3.000000,1.000000
75%,4.000000,3.000000,4.000000,3.000000,2.000000,2.000000,3.000000,3.000000,4.000000,3.000000,...,3.000000,4.000000,5.000000,5.000000,3.000000,4.000000,5.000000,4.000000,4.000000,2.000000
max,25.000000,17.000000,43.000000,24.000000,17.000000,30.000000,19.000000,48.000000,22.000000,22.000000,...,31.000000,25.000000,24.000000,20.000000,14.000000,49.000000,44.000000,19.000000,17.000000,49.000000


In [37]:
wind_direction.describe()

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,44458.000000,45252.000000,44459.000000,45252.000000,45252.000000,45252.000000,45248.000000,45252.000000,45252.000000,45252.000000,...,45252.000000,44460.000000,45252.000000,45252.000000,44460.000000,44460.000000,44456.000000,44460.000000,44460.000000,44460.000000
mean,159.892978,189.900071,209.299489,161.380226,139.549611,199.212852,176.320987,168.564351,199.878458,184.769005,...,202.947008,196.250247,190.133541,184.915562,218.891206,171.459199,204.244691,196.070558,194.901552,185.095592
std,98.163221,102.873761,95.589055,103.206816,105.591486,106.350252,106.902903,107.533094,104.462490,104.705030,...,102.323689,107.433676,99.033739,107.763589,102.304924,121.843329,144.891439,112.498247,111.983576,117.733600
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,80.000000,107.000000,160.000000,70.000000,29.000000,102.000000,94.000000,74.000000,125.000000,100.000000,...,120.000000,104.000000,111.000000,80.000000,112.000000,40.000000,40.000000,106.750000,102.000000,68.000000
50%,140.000000,198.000000,232.000000,175.000000,140.000000,230.000000,180.000000,180.000000,210.000000,200.000000,...,224.000000,220.000000,220.000000,210.000000,252.000000,184.000000,264.000000,224.000000,223.000000,230.000000
75%,236.000000,289.000000,280.000000,218.000000,230.000000,290.000000,255.000000,250.000000,292.000000,268.000000,...,289.000000,289.000000,260.000000,272.000000,309.000000,280.000000,345.000000,291.000000,290.000000,287.000000
max,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,...,360.000000,360.000000,360.000000,360.000000,359.000000,360.000000,360.000000,360.000000,360.000000,360.000000


In [39]:
# Function to create dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, 1])
    return np.array(X), np.array(Y)

In [40]:
look_back = 10
X, Y = create_dataset(scaled_data, look_back)

# Check for NaN values in the dataset
if np.isnan(X).any() or np.isnan(Y).any():
    print("NaN values found in the dataset. Please check the data processing steps.")
    exit()

# Reshape X to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

In [41]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [42]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [43]:
# Train the model
model.fit(X, Y, batch_size=1, epochs=1)


4514/4514 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - loss: 0.0429
